# Prediction Observation Classification Pipeline

- **Goal:** Prediction Classification

In [1]:
import os
import sys

import importlib.util

import pandas as pd


from pathlib import Path
from IPython.display import Image

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

import log_files
from data_processing import DataProcessing
from feature_extraction import TfidfFeatureExtraction, SpacyFeatureExtraction
# from classification_models import SkLearnPerceptronModel, SkLearnSGDClassifier, EvaluationMetric

# Absolute path to your local classification_models.py file
project_root = "/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions"
module_path = os.path.join(project_root, "classification_models.py")

# Dynamically load the module so it overrides any installed package
spec = importlib.util.spec_from_file_location("classification_models", module_path)
classification_models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(classification_models)

# Inject into sys.modules so pickle uses this
sys.modules["classification_models"] = classification_models

# ✅ Now you can use the classes
perception_model = classification_models.SkLearnPerceptronModel()
sgd_model = classification_models.SkLearnSGDClassifier()
EvaluationMetric = classification_models.EvaluationMetric
from classification_models import SkLearnPerceptronModel, SkLearnSGDClassifier, EvaluationMetric

In [2]:
# Image(filename='../misc_experiments/base_pipeline.png')

## 1-Data Acquisition

In [3]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [4]:
log_file_path = "data/prediction_logs"
predictions = True
predictions_df = log_files.read_data(notebook_dir, log_file_path, predictions)
len(predictions_df), predictions_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/prediction_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/prediction_logs/batch_1-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/prediction_logs/batch_1-prediction/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/prediction_logs/batch_2-prediction
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/prediction_logs/batch_2-prediction/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/prediction_logs/batch_3-pre

(628,
                                                                                                                    Base Sentence  \
 0                                     JPMorgan Chase forecasts that the net profit at Amazon potentially decrease in Q3 of 2027.   
 1                                  On August 21, 2024, Bank of America speculates the revenue at Microsoft will likely increase.   
 2                                                   Citigroup predicts on 2024-08-21, the operating income at Alphabet may rise.   
 3                              According to Goldman Sachs, the research and development expenses at Facebook would fall in 2025.   
 4  In 21 August 2024, Morgan Stanley envisions that the gross profit at Johnson & Johnson has some probability to remain stable.   
 5                                              The stock price at Visa should stay same in Q2 of 2026, according to Wells Fargo.   
 6                                           JPMorgan forecasts

In [5]:
log_file_path = "data/observation_logs"
predictions = False
observations_df = log_files.read_data(notebook_dir, log_file_path, predictions)
len(observations_df), observations_df.head(7)

Start logging batch
log_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/observation_logs
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/observation_logs/batch_1-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/observation_logs/batch_1-observation/batch_1-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/observation_logs/batch_2-observation
CSV to DF
Load saved csv: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/observation_logs/batch_2-observation/batch_2-from_df.csv
save_batch_directory: /Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/pipelines/../data/observation_logs/b

(1795,
                                                                                                                                Base Sentence  \
 0             The financial analyst at Goldman Sachs observed that the operating income at Tesla had increased in the first quarter of 2024.   
 1                                         On 2024-08-20 to 2025-08-20, Morgan Stanley speculates the stock price at Amazon will likely rise.   
 2                                                                       A young investor predicts on 2025-03-15, the S&P 500 index may rise.   
 3                                        According to Bank of America, the net profit at Microsoft would fall in the second quarter of 2026.   
 4  In 2027-01-01 to 2027-12-31, Wells Fargo envisions that the interest rates at the Federal Reserve have some probability to remain stable.   
 5             The trading volume at Apple should stay same in the fourth quarter of 2025, according to a financial expert 

## 2-Data Processing

In [6]:
pred_obs_dfs = [predictions_df, observations_df]
base_df = DataProcessing.concat_dfs(pred_obs_dfs)
shuffled_base_df = DataProcessing.shuffle_df(base_df)
len(shuffled_base_df), shuffled_base_df.head(7)

(2423,
                                                                                                                    Base Sentence  \
 0                  The policy expert, Dr. Linda Hart, saw that the unemployment rate in Detroit, MI had decreased in Q2 of 2028.   
 1  On 07/15/2022, the market researcher at TrendWatchers saw that the consumer confidence index at RetailMart would likely rise.   
 2                                            According to the local weather station, the humidity at Houston rose on 11/12/2022.   
 3                       According to the head of HR, the employee turnover rate at the company will fall in the next six months.   
 4                                         According to  Bloomberg, the interest rates at the Federal Reserve rose in Q3 of 2027.   
 5                                             On 09/15/2023, a financial expert saw the gross profit at Microsoft had increased.   
 6                                  In Q2 2026, Professor Pate

## 3-Feature Extraction

### TF x IDF

In [7]:
max_features = None

tf_idf_feature_extractor = TfidfFeatureExtraction(shuffled_base_df, 'Base Sentence')
tfidf_vectorized_features = tf_idf_feature_extractor.word_feature_extraction(max_features)
tfidf_vectorized_features_df = tf_idf_feature_extractor.feature_scores(max_features)
tfidf_vectorized_features_df.head(3)

,Base Sentence,Sentence Label,01,02,03,04,05,06,07,08,09,10,11,11th,12,14,14th,15,15th,16,17,18,19,1st,20,2012,2016,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2040,2045,2046,2048,2049,2050,2054,2065,21,22,23,24,25,25th,26,27,28,2nd,30,31,3rd,49ers,50,500,5g,76ers,abc,academic,academy,according,accuracy,accuweather,across,active,activity,adaptation,adherence,adler,administered,administration,administrative,adolescents,adoption,adults,advanced,advisers,advisor,advocacy,aerobic,affordable,africa,african,afternoon,aged,agencies,agency,ahmed,ai,aid,air,airport,airports,alex,algorithm,alibaba,alice,alicia,all,allergies,allocation,allowed,alphabet,amara,amazon,amelia,america,american,americans,among,an,analysis,analyst,analysts,and,anderson,andrew,angela,angeles,ann,anna,annual,anticipates,antonio,anxiety,anya,app,apple,applications,applying,approaching,approval,approvals,approved,april,arbor,arch,arctic,are,area,areas,arizona,armour,arrangements,arsenal,articles,artifacts,artificial,as,ashley,asia,asian,assembly,assist,assistance,assistant,assists,association,assurance,asthma,at,athletic,atlanta,atlantic,atmospheric,attempts,attendance,attending,aug,august,austin,australian,automotive,autumn,ava,average,averages,aviation,awarded,awareness,baker,balance,baltimore,bank,banker,banking,banks,bar,barcelona,barclays,barometric,based,bases,basketball,battery,batting,battleground,bay,bayern,be,beach,bears,been,beijing,beneficiaries,benjamin,bennett,berkeley,berkshire,best,between,biden,biotech,bipartisan,birth,bitcoin,blackhawks,blackrock,bleacher,block,blocked,blood,bloomberg,blue,bmi,bob,body,boeing,books,bookstore,borrowed,boston,botanical,boulder,brad,bradley,brand,braves,breaches,brian,bridge,broncos,brookings,brooks,brown,bucks,budget,buffett,building,buildright,bulls,bureau,business,businesses,buy,by,ca,cafeteria,cafeterias,café,cake,california,caloric,calorie,calvin,campaign,campaigns,campus,campuses,can,canada,cancer,canyon,capacity,capitalist,capitalization,capitol,captain,carbon,cardiovascular,care,career,carl,cars,carter,cases,cash,casino,catalyst,cato,cavaliers,cbs,cdc,celtics,census,center,centers,central,ceo,cessation,cfo,chains,chairman,chance,chances,change,changed,changi,changing,channel,charles,chase,chaser,chef,chen,chevron,chicago,chief,chiefs,childhood,children,chill,china,chocolate,cholesterol,chris,christine,chronic,citations,citibank,cities,citigroup,city,civic,claims,claire,clark,class,clean,cleveland,climate,climatologist,climb,climbed,clinic,clinics,clippers,clothing,cloud,club,cnn,coach,coalition,coast,coastal,coca,code,cola,college,colleges,colorado,coming,commentator,commerce,commercial,commission,commissioner,communities,community,companies,company,complaints,completion,compliance,composite,computer,conducted,confidence,congressional,consistent,construction,consultations,consulting,consumer,consumption,continued,control,conversion,coordinator,corner,corp,corporate,corporation,corporations,could,council,councils,count,counties,countries,county,course,coursera,courses,coverage,covid,cowboys,creation,creek,crime,crimes,cto,cubs,cuisine,culinary,current,customer,customers,cybersecurity,daily,dallas,daniel,data,datadive,dave,david,davis,de,deaths,debt,decade,december,decline,declined,decrease,decreased,decreases,defaults,defective,defense,defensive,deficit,demand,dengue,densely,density,denver,department,depression,desert,despite,detected,detravious,detroit,deutsche,developed,developing,development,devices,devils,dew,diabetes,diagnosis,diego,dietetics,diets,difference,differential,dig,digestive,dip,direct,director,discovered,disease,diseases,dishes,distribution,district,districts,dividend,documented,dodgers,doe,dolphins,dominating,donors,dorms,downtown,downward,dr,driven,drop,dropped,drug,due,duke,duration,during,eagles,early,earnings,eastern,eco,ecoclean,economic,economics,economist,economy,education,educational,effective,efficiency,efforts,elderly,elec

### POS and NER

In [8]:
# !python -m spacy download en_core_web_lg

In [9]:
pos_ner_feature_extractor = SpacyFeatureExtraction(shuffled_base_df, "Base Sentence")
sentences = DataProcessing.df_to_list(shuffled_base_df, 'Base Sentence')
word_leveL_disable_components = ["lemmatizer"]
all_pos_tags, word_to_pos_mapping, all_ner_entities, word_to_ner_mappings =  pos_ner_feature_extractor.extract_features(disable_components=word_leveL_disable_components)

Pipeline: ['tok2vec', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']


1it [00:00,  9.69it/s]

Spacy Doc (0):  The policy expert, Dr. Linda Hart, saw that the unemployment rate in Detroit, MI had decreased in Q2 of 2028.
Spacy Doc (1):  On 07/15/2022, the market researcher at TrendWatchers saw that the consumer confidence index at RetailMart would likely rise.
Spacy Doc (2):  According to the local weather station, the humidity at Houston rose on 11/12/2022.
Spacy Doc (3):  According to the head of HR, the employee turnover rate at the company will fall in the next six months.


2423it [00:04, 508.54it/s]


In [10]:
pos_df = DataProcessing.convert_to_df(all_pos_tags, mapping=word_to_pos_mapping)
ner_df = DataProcessing.convert_to_df(all_ner_entities, word_to_ner_mappings)
pos_ner_features = [pos_df, ner_df]
pos_ner_features_df = DataProcessing.concat_dfs(pos_ner_features, axis=1, ignore_index=False)
pos_ner_features_df.head(7)

2423it [00:01, 1808.19it/s]
2423it [00:00, 4784.04it/s]


,PRON_1,SPACE_3,ADJ_6,NOUN_3,NOUN_9,NUM_3,PUNCT_2,ADJ_3,ADP_7,PART_2,NOUN_5,PRON_2,PUNCT_1,NOUN_8,SYM_4,ADP_8,ADP_3,PROPN_6,PROPN_9,PROPN_3,DET_6,SPACE_1,PROPN_2,PART_1,NUM_1,SPACE_4,PUNCT_6,ADJ_5,NOUN_10,NUM_2,NUM_6,SCONJ_1,NOUN_6,VERB_4,VERB_1,NOUN_7,PUNCT_3,NUM_4,PROPN_8,ADJ_4,SYM_3,PROPN_1,PROPN_5,NOUN_2,SYM_1,DET_5,NUM_5,ADV_2,VERB_2,ADP_2,SYM_2,PROPN_10,ADP_9,ADP_5,CCONJ_2,DET_1,ADV_1,PROPN_4,AUX_3,SPACE_2,ADP_6,DET_3,SCONJ_2,DET_2,PUNCT_5,PUNCT_4,CCONJ_1,AUX_2,AUX_1,NOUN_4,ADP_4,NOUN_11,VERB_3,PROPN_7,ADJ_2,ADP_1,ADJ_1,DET_4,NOUN_1,GPE_2,CARDINAL_2,PERSON_1,DATE_1,DATE_2,ORDINAL_1,LAW_1,GPE_1,PERCENT_1,ORG_1,LOC_1,NORP_1,QUANTITY_1,MONEY_1,CARDINAL_1,ORG_2,PERSON_2,ORG_3,FAC_2,DATE_3,FAC_1,TIME_1,PRODUCT_1,ORG_4,EVENT_1
0,NaN,NaN,NaN,unemployment,NaN,NaN,",",NaN,NaN,NaN,NaN,NaN,",",NaN,NaN,NaN,of,Q2,NaN,Hart,NaN,NaN,Linda,NaN,2028,NaN,NaN,NaN,NaN,NaN,NaN,that,NaN,NaN,saw,NaN,",",NaN,NaN,NaN,NaN,Dr.,MI,expert,NaN,NaN,NaN,NaN,decreased,in,NaN,NaN,NaN,NaN,NaN,The,NaN,Detroit,NaN,NaN,NaN,NaN,NaN,the,NaN,.,NaN,NaN,had,rate,NaN,NaN,NaN,NaN,NaN,in,NaN,NaN,policy,NaN,NaN,Linda Hart,NaN,NaN,NaN,NaN,Detroit,NaN,MI,NaN,NaN,NaN,NaN,2028,Q2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,consumer,NaN,NaN,.,NaN,NaN,NaN,index,NaN,",",NaN,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,RetailMart,NaN,07/15/2022,NaN,NaN,NaN,NaN,NaN,NaN,that,NaN,NaN,saw,NaN,NaN,NaN,NaN,NaN,NaN,TrendWatchers,NaN,researcher,NaN,NaN,NaN,NaN,rise,at,NaN,NaN,NaN,NaN,NaN,the,likely,NaN,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,would,confidence,NaN,NaN,NaN,NaN,NaN,On,NaN,NaN,market,NaN,NaN,NaN,07/15/2022,NaN,NaN,NaN,NaN,NaN,TrendWatchers,NaN,NaN,NaN,NaN,NaN,RetailMart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,humidity,NaN,NaN,.,NaN,NaN,NaN,NaN,NaN,",",NaN,NaN,NaN,on,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11/12/2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,According,NaN,NaN,NaN,NaN,NaN,NaN,Houston,NaN,station,NaN,NaN,NaN,NaN,rose,at,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,to,local,NaN,weather,NaN,NaN,NaN,11/12/2022,NaN,NaN,NaN,Houston,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,turnover,NaN,NaN,.,NaN,NaN,NaN,company,NaN,",",NaN,NaN,NaN,at,NaN,NaN,NaN,NaN,NaN,NaN,NaN,six,NaN,NaN,NaN,NaN,NaN,NaN,NaN,months,NaN,According,NaN,NaN,NaN,NaN,NaN,NaN,HR,NaN,employee,NaN,NaN,NaN,NaN,fall,of,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,the,NaN,the,NaN,NaN,NaN,NaN,will,rate,in,NaN,NaN,NaN,NaN,to,next,the,head,NaN,NaN,NaN,the next six months,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN,NaN,",",NaN,NaN,NaN,in,NaN,NaN,Reserve,NaN,,Federal,NaN,2027,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,According,NaN,NaN,NaN,NaN,NaN,NaN,Bloomberg,NaN,rates,NaN,NaN,NaN,NaN,rose,at,NaN,NaN,NaN,NaN,NaN,the,NaN,Q3,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,of,NaN,NaN,NaN,NaN,to,NaN,NaN,interest,NaN,NaN,NaN,2027,NaN,NaN,NaN,Q3,NaN,Bloomberg,NaN,NaN,NaN,NaN,NaN,the Federal Reserve,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,.,NaN,NaN,NaN,NaN,NaN,",",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,09/15/2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,saw,NaN,NaN,NaN,NaN,NaN,NaN,Microsoft,NaN,profit,NaN,NaN,NaN,NaN,increased,at,NaN,NaN,NaN,NaN,NaN,a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,had,NaN,NaN,NaN,NaN,NaN,gross,On,financial,NaN,expert,NaN,NaN,NaN,09/15/2023,NaN,NaN,NaN,NaN,NaN,Microsoft,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,communities,NaN,NaN,.,NaN,NaN,NaN,NaN,NaN,",",NaN,NaN,NaN,NaN,NaN,NaN,Patel,NaN,NaN,Professor,NaN,2026,NaN,NaN,NaN,NaN,NaN,NaN,that,NaN,NaN,envisioned,NaN,NaN,NaN,NaN,NaN,NaN,Q2,NaN,rates,NaN,NaN,NaN,NaN,decreased,at,NaN,NaN,NaN,NaN,NaN,the,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,In,rural,NaN,smoking,NaN,NaN,Patel,Q2 2026,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
encoded_pos_ner_df = DataProcessing.encode_tags_entities_df(pos_ner_features_df, sentence_and_label_df=shuffled_base_df)
encoded_pos_ner_df.head(3)

,Base Sentence,Sentence Label,PRON_1,SPACE_3,ADJ_6,NOUN_3,NOUN_9,NUM_3,PUNCT_2,ADJ_3,ADP_7,PART_2,NOUN_5,PRON_2,PUNCT_1,NOUN_8,SYM_4,ADP_8,ADP_3,PROPN_6,PROPN_9,PROPN_3,DET_6,SPACE_1,PROPN_2,PART_1,NUM_1,SPACE_4,PUNCT_6,ADJ_5,NOUN_10,NUM_2,NUM_6,SCONJ_1,NOUN_6,VERB_4,VERB_1,NOUN_7,PUNCT_3,NUM_4,PROPN_8,ADJ_4,SYM_3,PROPN_1,PROPN_5,NOUN_2,SYM_1,DET_5,NUM_5,ADV_2,VERB_2,ADP_2,SYM_2,PROPN_10,ADP_9,ADP_5,CCONJ_2,DET_1,ADV_1,PROPN_4,AUX_3,SPACE_2,ADP_6,DET_3,SCONJ_2,DET_2,PUNCT_5,PUNCT_4,CCONJ_1,AUX_2,AUX_1,NOUN_4,ADP_4,NOUN_11,VERB_3,PROPN_7,ADJ_2,ADP_1,ADJ_1,DET_4,NOUN_1,GPE_2,CARDINAL_2,PERSON_1,DATE_1,DATE_2,ORDINAL_1,LAW_1,GPE_1,PERCENT_1,ORG_1,LOC_1,NORP_1,QUANTITY_1,MONEY_1,CARDINAL_1,ORG_2,PERSON_2,ORG_3,FAC_2,DATE_3,FAC_1,TIME_1,PRODUCT_1,ORG_4,EVENT_1
0,"The policy expert, Dr. Linda Hart, saw that the unemployment rate in Detroit, MI had decreased in Q2 of 2028.",0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0
1,"On 07/15/2022, the market researcher at TrendWatchers saw that the consumer confidence index at RetailMart would likely rise.",0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,"According to the local weather station, the humidity at Houston rose on 11/12/2022.",0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [12]:
comparison_result = (tfidf_vectorized_features_df.iloc[:, :2] == encoded_pos_ner_df.iloc[:, :2]).all().all()
comparison_result

np.True_

In [13]:
# If we don't (.iloc[:, 2:]), the sentence and label cols will be duplicated. Confirm that the first two columns are the same in both dataframes, so we can use the columns from the tfidf_vectorized_features_df dataframe.

updated_encoded_pos_ner_df = encoded_pos_ner_df.iloc[:, 2:]
sentence_label_features_df = DataProcessing.concat_dfs([tfidf_vectorized_features_df, updated_encoded_pos_ner_df], axis=1)
sentence_label_features_df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

## Models

1. Perceptron

---

- Split: 80% train and 20% test

In [14]:
# sentence_label_features_df[2]

In [15]:
X_train, X_test, y_train, y_test = DataProcessing.split_data(sentence_label_features_df, sentence_label_features_df[2])
X_train.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

In [16]:
def split_sentence_label_features(df: pd.DataFrame) -> tuple:
    sentences = df[0]
    prediction_labels = df[1]
    features_df = df.iloc[:, 2:]
    return sentences, prediction_labels, features_df

X_train_sentences, y_train_prediction_labels, X_train_features_df = split_sentence_label_features(X_train)

In [17]:
# y_train_prediction_labels

In [18]:
X_test_sentences, y_test_prediction_labels, X_test_features_df = split_sentence_label_features(X_test)
X_test_sentences[:3]

613     In 2024, Chef Daniel Lee envisions that the sales of gluten-free products at Whole Foods has some probability to remain stable.
1234                In January 2031, College Student Sarah Kim recorded that the win rate at the Philadelphia 76ers decreased slightly.
892                                      On June 10, 2026, the World Bank monitored the inflation rate in developing countries changed.
Name: 0, dtype: object

In [19]:
y_test_prediction_labels[:3]

613     0
1234    0
892     0
Name: 1, dtype: int64

In [20]:
perception_model = SkLearnPerceptronModel()
perception_model.train_model(X_train_features_df, y_train_prediction_labels)
perceptron_predictions = perception_model.predict(X_test_features_df)
perceptron_predictions.to_numpy().ravel()
perceptron_predictions[:3]

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


0    0
1    0
2    0
dtype: int64

In [21]:
sgd_model = SkLearnSGDClassifier()
sgd_model.train_model(X_train_features_df, y_train_prediction_labels)
sgd_predictions = sgd_model.predict(X_test_features_df)
sgd_predictions.to_numpy().ravel()
sgd_predictions[:3]

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


0    0
1    0
2    0
dtype: int64

In [22]:
model_predictions_df = pd.concat([X_test_sentences, y_test_prediction_labels], axis=1)
model_predictions_df.columns = ['Sentence', 'Actual Label']

model_predictions_df.head(3)

,Sentence,Actual Label
613,"In 2024, Chef Daniel Lee envisions that the sales of gluten-free products at Whole Foods has some probability to remain stable.",0
1234,"In January 2031, College Student Sarah Kim recorded that the win rate at the Philadelphia 76ers decreased slightly.",0
892,"On June 10, 2026, the World Bank monitored the inflation rate in developing countries changed.",0


In [23]:
model_predictions_df['Perceptron Predicted Label'] = perceptron_predictions.to_numpy().ravel()
model_predictions_df['SGD Predicted Label'] = sgd_predictions.to_numpy().ravel()
model_predictions_df.head(3)

,Sentence,Actual Label,Perceptron Predicted Label,SGD Predicted Label
613,"In 2024, Chef Daniel Lee envisions that the sales of gluten-free products at Whole Foods has some probability to remain stable.",0,0,0
1234,"In January 2031, College Student Sarah Kim recorded that the win rate at the Philadelphia 76ers decreased slightly.",0,0,0
892,"On June 10, 2026, the World Bank monitored the inflation rate in developing countries changed.",0,0,0


## Evaluation

In [24]:
get_metrics = EvaluationMetric()

In [25]:
metrics = get_metrics.eval_classification_report(y_test_prediction_labels, perceptron_predictions)
metrics

              precision    recall  f1-score   support

           0       0.97      0.89      0.93       343
           1       0.78      0.94      0.85       142

    accuracy                           0.90       485
   macro avg       0.87      0.91      0.89       485
weighted avg       0.91      0.90      0.91       485



In [26]:
metrics = get_metrics.eval_classification_report(y_test_prediction_labels, sgd_predictions)
metrics

              precision    recall  f1-score   support

           0       0.90      0.97      0.93       343
           1       0.91      0.75      0.82       142

    accuracy                           0.90       485
   macro avg       0.90      0.86      0.88       485
weighted avg       0.90      0.90      0.90       485



In [27]:
# import pickle
# base_dir = os.path.join(notebook_dir, '../models')
# sgd_path = os.path.join(base_dir, 'sgd_model.pkl')
# with open(sgd_path, 'wb') as file:
#     pickle.dump(sgd_model, file)

In [28]:
# sgd_path